We first import our new package: pandas

In [20]:
import pandas as pd

Pandas is very useful for working with real world data. Pandas dataframes are a bit like excel spreadsheets, but optimized to be used efficiently in python.

Next, we would therefore usually read in our data into a dataframe. The most common way to do this is from a .csv file. 

In [ ]:
pd.read_csv("C:\\Users\\...\\nameoffile.csv", sep=",", decimal=".")

The first argument of this pandas function is your csv file (with the correct path to it). With the sep (seperator) and decimal arguments, you can tell pandas how your csv file is formated. This depends on how you saved the csv file (e.g. with an English or German version of excel). If the numbers in the csv file are seperated by commas, and the decimal sign within a number is a period, then the above code is what you should use. If, instead, decimals are denoted by commas and the different numbers in your csv file are seprated by semicolons, then you should instead write: 

In [ ]:
pd.read_csv("C:\\Users\\...\\nameoffile.csv", sep=";", decimal=",")

In the assigment you will read-in data files, as above. However, in this lecture we stick with a simple example that I just create. I can for example create a dataframe out of a numpy array. However in a pandas data frame, all the columns have names, so I should pass these on:

In [21]:
import numpy as np 
df = pd.DataFrame(np.array([['2001Q1', 2, 10], ['2001Q2', 3, 20], ['2001Q3', 4, 30],['2001Q4', 5, 40]]),
                   columns=['a', 'b', 'c'])
df

,a,b,c
0,2001Q1,2,10
1,2001Q2,3,20
2,2001Q3,4,30
3,2001Q4,5,40


The content of column 'a' I want to represent different quarters. However, pandas does not automatically recognize this column as a time/date index, and I should explicetly tell this to pandas as follows:

In [22]:
df['a']=pd.to_datetime(df['a'])
df

,a,b,c
0,2001-01-01,2,10
1,2001-04-01,3,20
2,2001-07-01,4,30
3,2001-10-01,5,40


Now we see that pandas recognized the column as dates and put the column into the default date formatting. I don't like this formatting and want the notation with quarters back. So let's set the frequency I  want pandas to use as the period-unit to quarters.

In [23]:
df['a']=pd.PeriodIndex(df['a'],freq='Q')
df

,a,b,c
0,2001Q1,2,10
1,2001Q2,3,20
2,2001Q3,4,30
3,2001Q4,5,40


This dataframe looks the same as 2 steps before, but the difference is that column 'a' is now recognized as a date, so that we can use this date as the index of the dataframe. This will be more meaningful than 0,1,2... 
We can do this with the set_index method. This method is applied to a whole dataframe (not to a column of it):

In [24]:
df=df.set_index(['a'])
df

,b,c
a,,
2001Q1,2,10
2001Q2,3,20
2001Q3,4,30
2001Q4,5,40


Now the meaningless index has disappeared and our column 'a' is no longer a data column, but instead an index column. We see this from 'a' being printed lower than 'b' and 'c'.

Before we can do any data analysis with  our new dataframe, we must be sure that 
our columns with data are recognized as floating numbers (or integers would also be fine here). In this example (where I created the dataframe out of a numpy array that had a mixture of strings and numbers) they are not. We can check this as follows.

In [25]:
df['b'].dtypes

dtype('O')

O means type 'object', which is not what we want. Let's therefore change the types of our two columns to float.

In [26]:
df['b']=df['b'].astype(float)
df['c']=df['c'].astype(float)
df.dtypes

b    float64
c    float64
dtype: object

Another thing that I want to introduce that is related to the above is how to change the frequency of the data by averaging. For example, suppose that we just want one observation per year rather than one observation per quarter (and hence four per year).
Then we can first set the frequency of the time index to yearly and then merge all entries with the same index into one number with the groupby method. If we want the merging method to be averaging, we should add the mean method to the groupby method as follows:

In [27]:
df_yearly=df.copy()
df_yearly.index=pd.PeriodIndex(df_yearly.index, freq='Y')
df_yearly

,b,c
a,,
2001,2.0,10.0
2001,3.0,20.0
2001,4.0,30.0
2001,5.0,40.0


In [28]:
df_yearly=df.groupby(df_yearly.index).mean() 
df_yearly

,b,c
a,,
2001,3.5,25.0


Now we have a new dataframe which I gave a different name, that has only one observation per year and that is obtained by averaging over all quarters within a year.

We will stick in this example with the quarterly data in df, though. Lets now use our datafrane for some analysis. In economic applications we will often want to do a linear regression. For example, we want to see if the time series of the b variable  can be explained by the time sereis of the c variable.

There are different python packages that can be used for statistical analysis and that support working with pandas dataframes. One of these pakcages that is used a lot is statsmodels. We will not do a much with that package in this course, but let's just see how we can do an OLS regression with the ols function that we import from statsmodels.

We then first need to save the output of the ols function as a variable. This is our - not yet estimated - regression model. As the first argument ‘formula’ in the ols function we can give the regression equation. This can be done with intuitive syntax in a string. First write your dependent variable, then the ~ sign and then your regressors separated by + signs. As the second argument of the ols function, ‘data’, we need to give the name of our pandas dataframe where our variables are stored. The names of the dapendent variable and regressors that we give in the formula string must correspond to the names of the columns our dataframe. 

Next, we fit the model that we saved with the fit method. This applies the ols estimation. 

Finally we can see the results with the summary method.

In [29]:
from statsmodels.formula.api import ols
mod = ols(formula='b ~  c', data=df)
fit = mod.fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      b   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.014e+31
Date:                Sun, 24 Jan 2021   Prob (F-statistic):           9.86e-32
Time:                        18:41:50   Log-Likelihood:                 135.28
No. Observations:                   4   AIC:                            -266.6
Df Residuals:                       2   BIC:                            -267.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0000    8.6e-16   1.16e+15      0.0

C:\Users\joepl\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)


As you see, we found a clear realtionshsip where it holds that bt =1+0.1*ct.

But what if we instead want to see if b depends on the lagged value of c:  b_t = alpha +beta* c_{t-1} ?

Regressions based on pandas data frames will always take together the data contained in a particular row of the data frame. Hence, if we want to include the lagged value of c in a regression together with the current value of b, we must make sure that the lagged value of c shows up in the same row as the current value of b. Let's, therefore, create a new column c, that is shifted 1 entry downward comared to the original c column. If we do that, then the lagged value will always show up in the row where normally the current value would have shown up (you will see this below). We can use the shift method for this, where in the argument of the shift method we can say how many rows we want to shift the column downward.

In [30]:
df['c_lag']=df['c'].shift(+1)
df

,b,c,c_lag
a,,,
2001Q1,2.0,10.0,NaN
2001Q2,3.0,20.0,10.0
2001Q3,4.0,30.0,20.0
2001Q4,5.0,40.0,30.0


As you see, the new c_lag column always show the lagged value of c in the row where the current value of b is located. This is what we wanted. So now we do a regression of b on the new c_lag column.

In [31]:
from statsmodels.formula.api import ols
mod = ols(formula='b ~  c_lag', data=df)
fit = mod.fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      b   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.451e+29
Date:                Sun, 24 Jan 2021   Prob (F-statistic):           6.93e-16
Time:                        18:41:58   Log-Likelihood:                 99.715
No. Observations:                   3   AIC:                            -195.4
Df Residuals:                       1   BIC:                            -197.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0000   2.35e-15   8.51e+14      0.0

C:\Users\joepl\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


We find that bt=2+0.1*c{t-1}

And what if we have 2 data frames?

We can combine them, for example with the concat function. Either vertically  (axis=0), if the data frames contain different data point of the same variables

In [35]:
df2 = pd.DataFrame(np.array([['2002Q1', 12, 12], ['2002Q2', 13, 22], ['2002Q3', 24, 31],['2002Q4', 35, 4]]),
                   columns=['a', 'b', 'c'])
df2['a']=pd.to_datetime(df2['a'])
df2['a']=pd.PeriodIndex(df2['a'],freq='Q')
df2=df2.set_index(['a'])
df2

,b,c
a,,
2002Q1,12,12
2002Q2,13,22
2002Q3,24,31
2002Q4,35,4


In [36]:
pd.concat([df,df2],axis=0)

,b,c,c_lag
a,,,
2001Q1,2,10,NaN
2001Q2,3,20,10.0
2001Q3,4,30,20.0
2001Q4,5,40,30.0
2002Q1,12,12,NaN
2002Q2,13,22,NaN
2002Q3,24,31,NaN
2002Q4,35,4,NaN


Or horizonzally (axis=1), if  we have addtional variables that cover the same time periods:

In [37]:
df2 = pd.DataFrame(np.array([['2001Q1', 12, 12], ['2001Q2', 13, 22], ['2001Q3', 24, 31],['2001Q4', 35, 4]]),
                   columns=['a', 'd', 'e'])
df2['a']=pd.to_datetime(df2['a'])
df2['a']=pd.PeriodIndex(df2['a'],freq='Q')
df2=df2.set_index(['a'])
df2

,d,e
a,,
2001Q1,12,12
2001Q2,13,22
2001Q3,24,31
2001Q4,35,4


In [38]:
pd.concat([df,df2],axis=1)

,b,c,c_lag,d,e
a,,,,,
2001Q1,2.0,10.0,NaN,12,12
2001Q2,3.0,20.0,10.0,13,22
2001Q3,4.0,30.0,20.0,24,31
2001Q4,5.0,40.0,30.0,35,4
